# Changes description compared to 0.1
- 2 experiments instead of 1: we switch the sides of attackers and benign traffic generator to mitigate TTL issue

## Collecting data from Campus infrastructure

In [ ]:
import sys
sys.version

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import os

from netunicorn.base import MinionPool, Task
from netunicorn.base.experiment import Experiment, ExperimentStatus
from netunicorn.base.pipeline import Pipeline
from netunicorn.library.basic import ShellCommand
from netunicorn.client.remote import RemoteClient

In [ ]:
experiment_label_1 = 'patator-0.2.4-1'
experiment_label_2 = 'patator-0.2.4-2'

In [ ]:
class BenignTask(Task):
    def __init__(self, url):
        self.url = url
    
    def run(self):
        import subprocess
        for i in range(500):
            subprocess.check_output(f'curl {self.url}', shell=True)
        return 0

In [ ]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['VICTIM_ENDPOINT']} "
        "user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

In [ ]:
benign_pipeline = Pipeline()
benign_pipeline.then(BenignTask('redacted'))
benign_pipeline.then(BenignTask('redacted'))

benign_pipeline.environment_definition.image = "redacted"

In [ ]:
client = RemoteClient(
    endpoint=os.environ['UNICORN_ENDPOINT'],
    login=os.environ['UNICORN_LOGIN'],
    password=os.environ['UNICORN_PASSWORD']
)

In [ ]:
minion_pool = client.get_minion_pool()

In [ ]:
pool_1 = MinionPool([x for x in minion_pool if x.name in {
    'raspi-e4:5f:01:56:d9:a3',
    'raspi-e4:5f:01:75:6b:2c',
    'raspi-e4:5f:01:72:a2:eb',
    'raspi-e4:5f:01:56:d8:fc',
}])

pool_2 = MinionPool([x for x in minion_pool if x.name in {
    'raspi-dc:a6:32:d7:6e:64',
    'raspi-e4:5f:01:56:d9:73',
    'raspi-e4:5f:01:56:d9:8b'
}])

In [ ]:
experiment_1 = Experiment().map(pool_1, attacker_pipeline).map(pool_2, benign_pipeline)
experiment_2 = Experiment().map(pool_2, attacker_pipeline).map(pool_1, benign_pipeline)

In [ ]:
client.prepare_experiment(experiment_1, experiment_label_1)

In [ ]:
client.prepare_experiment(experiment_2, experiment_label_2)

In [ ]:
client.start_execution(experiment_label_1)

In [ ]:
client.start_execution(experiment_label_2)

## Collecting from Azure infrastructure

In [ ]:
experiment_label_1 = 'patator-0.2.1-azure-1'
experiment_label_2 = 'patator-0.2.1-azure-2'

In [ ]:
benign_pipeline = Pipeline()
benign_pipeline.then(BenignTask('http://172.174.8.19/about.html'))
benign_pipeline.then(BenignTask('http://172.174.8.19:26511/health'))

benign_pipeline.environment_definition.image = "redacted"

In [ ]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['AZURE_VICTIM_ENDPOINT']} "
        "user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

In [ ]:
experiment_1 = Experiment().map(pool_1, attacker_pipeline).map(pool_2, benign_pipeline)
experiment_2 = Experiment().map(pool_2, attacker_pipeline).map(pool_1, benign_pipeline)

In [ ]:
client.prepare_experiment(experiment_1, experiment_label_1)

In [ ]:
client.prepare_experiment(experiment_2, experiment_label_2)

In [ ]:
client.start_execution(experiment_label_1)

In [ ]:
client.start_execution(experiment_label_2)

## Collecting from multicloud infrastructure

In [ ]:
pool_1 = MinionPool([x for x in minion_pool if x.name in {
    'netunicorn-search-aws-1',
    'netunicorn-search-digitalocean-1',
}])
pool_2 = MinionPool([x for x in minion_pool if x.name in {
    'netunicorn-search-aws-2',
    'netunicorn-search-aws-3',
    'netunicorn-search-digitalocean-2',
}])

In [ ]:
assert len(pool_1) == 2
assert len(pool_2) == 3

In [ ]:
experiment_label_1 = 'patator-0.2.6-multicloud-1'
experiment_label_2 = 'patator-0.2.6-multicloud-2'

In [ ]:
benign_pipeline = Pipeline()
benign_pipeline.then(BenignTask('http://172.174.8.19/about.html'))
benign_pipeline.then(BenignTask('http://172.174.8.19:26511/health'))

benign_pipeline.environment_definition.image = "redacted"

In [ ]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['AZURE_VICTIM_ENDPOINT']} "
        "user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

In [ ]:
experiment_1 = Experiment().map(pool_1, attacker_pipeline).map(pool_2, benign_pipeline)
experiment_2 = Experiment().map(pool_2, attacker_pipeline).map(pool_1, benign_pipeline)

In [ ]:
client.prepare_experiment(experiment_1, experiment_label_1)

In [ ]:
client.prepare_experiment(experiment_2, experiment_label_2)

In [ ]:
client.start_execution(experiment_label_1)

In [ ]:
client.start_execution(experiment_label_2)